The aim of this notebook is to illustrate the usage of *wnpoly*.  Begin by installing the package.

In [1]:
import sys, subprocess, pkg_resources
required = {'numpy','wnpoly'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    subprocess.check_call([sys.executable,'-m','pip','install','--quiet',*missing])

Next import the packages.

In [2]:
import numpy as np
import wnpoly as wp

*wnpoly* currently has two modules.  The first handles [Symmetric polynomials](#symmetric), which are polynomials that remain unchanged under exchange of any variables.  This [Wikipedia page](https://en.wikipedia.org/wiki/Symmetric_polynomial) provides an introduction.  The second handles [Bell polynomials](#bell), which appear particularly in problems involving [set partitions](https://en.wikipedia.org/wiki/Bell_polynomials).

<a id='symmetric'></a>
# Symmetric polynomials

This section illustrates *wnpoly's* symmetric polynomial classes, which include [complete homogeneous symmetric](https://en.wikipedia.org/wiki/Complete_homogeneous_symmetric_polynomial), [elementary homogeneous symmetric](https://en.wikipedia.org/wiki/Elementary_symmetric_polynomial), and [power sum symmetric](https://en.wikipedia.org/wiki/Power_sum_symmetric_polynomial) polynomials.

Begin by creating instances of each class.

In [3]:
chsp = wp.symm.complete()
ehsp = wp.symm.elementary()
pssp = wp.symm.power_sum()

Now create a set of variables *x*.  Here we choose a set of ten randomly chosen integers between 1 and 10.  By definition, *x[0]* should equal 0, so we insert that at the beginning of the array; thus, the array has 11 elements with index ranging from 0 to 10.  In fact, the *wnpoly* routines do not use *x[0]* in computations but rather use it as a placeholder to allow the index of the array to correspond to the expected *x* or polynomial.

In [4]:
n = 10
x = np.random.default_rng().integers(1, high=10, size=n)
x = np.insert(x, 0, 0)

Now compute the corresponding set of polynomials of the various types up to order *n*.

In [5]:
h = chsp.compute(x, n)
e = ehsp.compute(x, n)
p = pssp.compute(x, n)

Print out the values and the *x*'s.

In [6]:
for n in range(len(x)):
    print("n = {:d}, x[n] = {:d}, h[n] = {:d}, e[n] = {:d}, p[n] = {:d}".format(n, x[n], h[n], e[n], p[n]))

n = 0, x[n] = 0, h[n] = 1, e[n] = 1, p[n] = 1
n = 1, x[n] = 4, h[n] = 48, e[n] = 48, p[n] = 48
n = 2, x[n] = 3, h[n] = 1284, e[n] = 1020, p[n] = 264
n = 3, x[n] = 3, h[n] = 25308, e[n] = 12636, p[n] = 1620
n = 4, x[n] = 3, h[n] = 410562, e[n] = 101070, p[n] = 10728
n = 5, x[n] = 7, h[n] = 5811588, e[n] = 545508, p[n] = 74628
n = 6, x[n] = 5, h[n] = 74372692, e[n] = 2012684, p[n] = 535704
n = 7, x[n] = 8, h[n] = 880889268, e[n] = 5014500, p[n] = 3925620
n = 8, x[n] = 5, h[n] = 9816312027, e[n] = 8077833, p[n] = 29179848
n = 9, x[n] = 7, h[n] = 104158393308, e[n] = 7601580, p[n] = 219172068
n = 10, x[n] = 3, h[n] = 1061866328616, e[n] = 3175200, p[n] = 1659508344


We can now check some of [Newton's identities](https://en.wikipedia.org/wiki/Newton%27s_identities).  The first we will check relates the complete and elementary polynomials such that $d_n = \sum_{k = 0}^n (-1)^k h_{n-k}(x_1, ..., x_n) e_k(x_1, ..., x_n) = 0$.

In [7]:
for n in range(1, len(x)):
    d = 0
    for k in range(n+1):
        d += np.power(-1, k) * h[n-k] * e[k]
    print("n = {:d}, d[n] = {:d}".format(n, d))

n = 1, d[n] = 0
n = 2, d[n] = 0
n = 3, d[n] = 0
n = 4, d[n] = 0
n = 5, d[n] = 0
n = 6, d[n] = 0
n = 7, d[n] = 0
n = 8, d[n] = 0
n = 9, d[n] = 0
n = 10, d[n] = 0


Now we check an identity relating the complete homogeneous symmetric and power sum symmetric polynomials, namely, that $nh_n = \sum_{k=1}^n h_{n-k} p_k$.

In [8]:
for n in range(1, len(x)):
    sum = 0
    for k in range(1, n+1):
        sum += h[n-k] * p[k]
    print(
        "n = {:d}, n*h[n] = {:d}, sum = {:d}, difference = {:d}".format(
            n, n*h[n], sum, n*h[n] - sum)
    )

n = 1, n*h[n] = 48, sum = 48, difference = 0
n = 2, n*h[n] = 2568, sum = 2568, difference = 0
n = 3, n*h[n] = 75924, sum = 75924, difference = 0
n = 4, n*h[n] = 1642248, sum = 1642248, difference = 0
n = 5, n*h[n] = 29057940, sum = 29057940, difference = 0
n = 6, n*h[n] = 446236152, sum = 446236152, difference = 0
n = 7, n*h[n] = 6166224876, sum = 6166224876, difference = 0
n = 8, n*h[n] = 78530496216, sum = 78530496216, difference = 0
n = 9, n*h[n] = 937425539772, sum = 937425539772, difference = 0
n = 10, n*h[n] = 10618663286160, sum = 10618663286160, difference = 0


Finally, we will check the identity that $n e_n = \sum_{k=1}^n (-1)^{k-1} e_{n-k} p_k$.

In [9]:
for n in range(1, len(x)):
    sum = 0
    for k in range(1, n+1):
        sum += np.power(-1, k-1) * e[n-k] * p[k]
    print(
        "n = {:d}, n*e[n] = {:d}, sum = {:d}, difference = {:d}".format(
            n, n*e[n], sum, n*e[n] - sum)
    )

n = 1, n*e[n] = 48, sum = 48, difference = 0
n = 2, n*e[n] = 2040, sum = 2040, difference = 0
n = 3, n*e[n] = 37908, sum = 37908, difference = 0
n = 4, n*e[n] = 404280, sum = 404280, difference = 0
n = 5, n*e[n] = 2727540, sum = 2727540, difference = 0
n = 6, n*e[n] = 12076104, sum = 12076104, difference = 0
n = 7, n*e[n] = 35101500, sum = 35101500, difference = 0
n = 8, n*e[n] = 64622664, sum = 64622664, difference = 0
n = 9, n*e[n] = 68414220, sum = 68414220, difference = 0
n = 10, n*e[n] = 31752000, sum = 31752000, difference = 0


It is also possible to compute the symmetric polynomials normalized by the number of terms in each polynomial.  Here one should not use integers but rather floats or similar type.  Again, the first *x* should be zero, so insert that.

In [10]:
n = 5
x = np.random.default_rng().uniform(0.0, 10.0, size=n)
x = np.insert(x, 0, 0.)

Now compute and print out the normalized polynomials.

In [11]:
h_norm = chsp.compute_normalized(x, n+1)
e_norm = ehsp.compute_normalized(x, n+1)
p_norm = pssp.compute_normalized(x, n+1)

for n in range(len(x)):
    print(
        "n = {:d}, x[n] = {:.4f}, h_norm[n] = {:.4f}, e_norm[n] = {:.4f}, p_norm[n] = {:.4f}".format(
        n, x[n], h_norm[n], e_norm[n], p_norm[n]))

n = 0, x[n] = 0.0000, h_norm[n] = 1.0000, e_norm[n] = 1.0000, p_norm[n] = 1.0000
n = 1, x[n] = 0.3789, h_norm[n] = 2.0675, e_norm[n] = 2.0675, p_norm[n] = 2.4810
n = 2, x[n] = 5.1817, h_norm[n] = 4.8145, e_norm[n] = 3.5183, p_norm[n] = 9.6659
n = 3, x[n] = 0.8778, h_norm[n] = 12.3299, e_norm[n] = 4.5477, p_norm[n] = 43.5427
n = 4, x[n] = 1.8073, h_norm[n] = 34.0946, e_norm[n] = 4.1152, p_norm[n] = 206.2939
n = 5, x[n] = 4.1592, h_norm[n] = 100.3710, e_norm[n] = 2.1588, p_norm[n] = 1000.0296


<a id='bell'></a>
# Bell polynomials

This section illustrates *wnpoly's* Bell polynomial module.  The module has a class for complete and partial Bell polynomials.

Begin by creating an instance of the Bell polynomial class.

In [12]:
my_bell = wp.bell.bell()

Now generate some Bell polynomials.  By definition, $B_0 = 1$, so insert that.

In [13]:
n = 5
b = np.random.default_rng().uniform(low=1, high=10, size=n)
b = np.insert(b, 0, 1.)

Now invert the Bell polynomials to find the *x's* that would give rise to those Bell polynomials.

In [14]:
x = my_bell.invert(b)

Compute the Bell polynomials from those *x's* and use them to compute the Bell polynomials.  Compare the computed polynomials to the original ones.

In [15]:
bc = my_bell.compute(x)

for n in range(len(x)):
    print(
        "n = {:d}, x[n] = {:.6e}, b[n] = {:.6e}, bc[n] = {:.6e}".format(
            n, x[n], b[n], bc[n]
        )
    )

n = 0, x[n] = 0.000000e+00, b[n] = 1.000000e+00, bc[n] = 1.000000e+00
n = 1, x[n] = 8.536660e+00, b[n] = 8.536660e+00, bc[n] = 8.536660e+00
n = 2, x[n] = -6.787997e+01, b[n] = 4.994594e+00, bc[n] = 4.994594e+00
n = 3, x[n] = 1.121578e+03, b[n] = 5.278757e+00, bc[n] = 5.278757e+00
n = 4, x[n] = -2.774324e+04, b[n] = 8.323257e+00, bc[n] = 8.323257e+00
n = 5, x[n] = 9.150962e+05, b[n] = 6.641347e+00, bc[n] = 6.641347e+00


Now we study the partial Bell polynomial class.  First create an instance.

In [16]:
my_partial_bell = wp.bell.partial_bell()

Now compute the partial Bell polynomials from the *x's* above.  The return is a two-dimensional array with the indices *n* and *k* giving $B_{n,k}(x_1, ..., x_{n-k+1})$.

In [17]:
pbc = my_partial_bell.compute(x)

Check the results by noting that the complete Bell polynomials $B_n$ are given by $B_n(x_1, ..., x_n) = \sum_{k=1}^n B_{n,k}(x_1, ..., x_{n-k+1})$.  We note that $B_{0,0} = 1$ but $B_{n,0} = 0$ for $n > 0$.

In [18]:
for n in range(len(x)):
    bn = 0
    for k in range(0, n+1):
        bn += pbc[n, k]
    print(
        "n = {:d}, x[n] = {:.6e}, bc[n] = {:.6e}, bn = {:.6e}".format(
            n, x[n], bc[n], bn
        )
    )

n = 0, x[n] = 0.000000e+00, bc[n] = 1.000000e+00, bn = 1.000000e+00
n = 1, x[n] = 8.536660e+00, bc[n] = 8.536660e+00, bn = 8.536660e+00
n = 2, x[n] = -6.787997e+01, bc[n] = 4.994594e+00, bn = 4.994594e+00
n = 3, x[n] = 1.121578e+03, bc[n] = 5.278757e+00, bn = 5.278757e+00
n = 4, x[n] = -2.774324e+04, bc[n] = 8.323257e+00, bn = 8.323257e+00
n = 5, x[n] = 9.150962e+05, bc[n] = 6.641347e+00, bn = 6.641347e+00


As a final exercise, we note that $x_n = B_{n,1}$.  Here we check that.

In [19]:
for n in range(len(x)):
    print("n = {:d}, x[n] = {:.6e}, pbc[n,1] = {:.6e}".format(n, x[n], pbc[n, 1]))

n = 0, x[n] = 0.000000e+00, pbc[n,1] = 0.000000e+00
n = 1, x[n] = 8.536660e+00, pbc[n,1] = 8.536660e+00
n = 2, x[n] = -6.787997e+01, pbc[n,1] = -6.787997e+01
n = 3, x[n] = 1.121578e+03, pbc[n,1] = 1.121578e+03
n = 4, x[n] = -2.774324e+04, pbc[n,1] = -2.774324e+04
n = 5, x[n] = 9.150962e+05, pbc[n,1] = 9.150962e+05
